In [1]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import scipy
from fooof import FOOOF
from lagged_autocoherence import *
import mne

In [2]:
freq_lims = [5,100]
lags = np.arange(1,4,.1)

def many_is_in(multiple, target):
    check_ = []
    for i in multiple:
        check_.append(i in target)
    return any(check_)

sens = ["MLC1", "MLC25", "MLC32", "MLC42", "MLC54", "MLC55", "MLC63"]

base_dir='/home/common/mszul/explicit_implicit_beta/derivatives/processed/'
out_dir='/home/bonaiuto/lagged_hilbert_coherence/output/explicit-implicit/'

In [ ]:
sub_paths=sorted(glob.glob(os.path.join(base_dir,'sub-*')))
for sub_idx, sub_path in enumerate(sub_paths):
    sub=os.path.split(sub_path)[-1]
    sub_out_path=os.path.join(out_dir,sub)
    if not os.path.exists(sub_out_path):
        os.mkdir(sub_out_path)

    epo_paths=sorted(glob.glob(os.path.join(sub_path,'{}-*-epo.fif'.format(sub))))
    for epo_path in epo_paths:
        block_num=os.path.split(epo_path)[-1].split('-')[2]
        epo_type=os.path.split(epo_path)[-1].split('-')[3]
        if epo_type=='visual' or epo_type=='motor':
            epo = mne.read_epochs(epo_path)
            epo = epo.pick_types(meg=True, ref_meg=False,misc=False)

            channels_used = [i for i in epo.info.ch_names if many_is_in(["MLC"], i)]
            channels_used = [i for i in channels_used if not many_is_in(sens, i)]

            fs=epo.info['sfreq']
            epo_data=epo.get_data()
            epo_data=np.moveaxis(epo_data,[0,1,2],[1,0,2])

            psd=[]
            lc=[]
            chs=[]
            for ch_idx in range(epo_data.shape[0]):
                if epo.info['ch_names'][ch_idx] in channels_used:
                    data=epo_data[ch_idx,:,:]
                    chs.append(ch_idx)

                    freqs,ch_psd=scipy.signal.welch(data, fs=fs, window='hann', 
                                                nperseg=int(fs), noverlap=int(fs/2), nfft=int(fs*2), detrend='constant', 
                                                return_onesided=True, scaling='density', axis=- 1, average='mean')
                    f_idx=(freqs>=freq_lims[0]) & (freqs<=freq_lims[-1])
                    freqs=freqs[f_idx]
                    ch_psd=np.mean(ch_psd[:,f_idx],axis=0)

                    ch_lc=lagged_hilbert_autocoherence(data, freqs, lags, fs, surr_method='phase', n_jobs=20)

                    psd.append(ch_psd)
                    lc.append(ch_lc)
            chs=np.array(chs)
            psd=np.array(psd)
            lc=np.array(lc)

            out_fname=os.path.join(sub_out_path,'{}-{}-{}.npz'.format(sub,block_num,epo_type))
            np.savez(out_fname, 
                     **{          
                         'chs': chs,
                         'freqs': freqs,
                         'lags': lags,
                         'psd': psd,
                         'lc': lc
                        })

Reading /home/common/mszul/explicit_implicit_beta/derivatives/processed/sub-101/sub-101-002-motor-epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        5 CTF compensation matrices available
Not setting metadata
56 matching events found
No baseline correction applied
0 projection items activated
Removing 5 compensators from info because not all compensation channels were picked.


Exception in thread ExecutorManagerThread:
Traceback (most recent call last):
  File "/home/bonaiuto/miniconda3/envs/new_lc/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/bonaiuto/miniconda3/envs/new_lc/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 566, in run
    self.terminate_broken(bpe)
  File "/home/bonaiuto/miniconda3/envs/new_lc/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 750, in terminate_broken
    self.kill_workers(reason="broken executor")
  File "/home/bonaiuto/miniconda3/envs/new_lc/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 781, in kill_workers
    recursive_terminate(p)
  File "/home/bonaiuto/miniconda3/envs/new_lc/lib/python3.10/site-packages/joblib/externals/loky/backend/utils.py", line 28, in recursive_terminate
    _recursive_terminate_without_psutil(process)
  File "/home/bonaiuto/miniconda3/envs/new_lc/lib/python3.10/site-p